#### Zbiór bardzo ciekawy, ale udostępniony w dziwnej formie (ogromny graf .ttl albo kulawy 'explorer' online z ograniczonym exportem) i mający dużo duplikatów. 
- usunąłem duplikaty
- usunąłem teksty w językach innych niż EN (automatyczna detekcja 'langdetect', mam nadzieję, że zadziałała do końca)
- usunąłem niewłaściwie sformułowane teksty z kategorii 'fake' (interpretacja zamiast rozpatrywanego twierdzenia)
- Ostatecznie udało się wyciągnąć około 20K unikalnych wpisów, w miarę zbalansowane i wg mnie ciekawe
- Teksty są przeważnie KRÓTKIE, ale to nie są nagłówki - raczej treściwe "newsy" w paru słowach
- Z kolei kolumna "headlines" z tego zbioru była bezużyteczna, ponieważ są to tytuły "recenzji" z fact-checkerów - zupełnie co innego i często zdradzające werdykt

In [2]:
import pandas as pd
import re
import nltk
import warnings
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline    
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, classification_report, accuracy_score, f1_score

In [3]:
df = pd.read_csv("ClaimsKG dataset\ClaimsKG_SPARQL_query_results_ALL.csv", on_bad_lines="warn")
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\local\AppData\Local\Temp\ipykernel_8692\1510327636.py:1: SyntaxWarning: invalid escape sequence '\C'
  df = pd.read_csv("ClaimsKG dataset\ClaimsKG_SPARQL_query_results_ALL.csv", on_bad_lines="warn")


,text,label,source
0,"While serving as U.S. Vice President, Henry A....",True,snopes
1,A 1964 campaign ad for President Lyndon B. Joh...,True,snopes
2,Fifty Republican former national security offi...,True,snopes
3,Elon Musk deleted a tweet where he called the ...,True,snopes
4,Texas Lt. Gov. Dan Patrick said COVID-19 socia...,True,snopes


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82654 entries, 0 to 82653
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    82654 non-null  object
 1   label   82654 non-null  bool  
 2   source  82654 non-null  object
dtypes: bool(1), object(2)
memory usage: 1.3+ MB


In [5]:
df = df.drop_duplicates(keep='first', ignore_index=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23083 entries, 0 to 23082
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    23083 non-null  object
 1   label   23083 non-null  bool  
 2   source  23083 non-null  object
dtypes: bool(1), object(2)
memory usage: 383.3+ KB


In [26]:
df.isnull().sum()

text      0
label     0
source    0
dtype: int64

In [28]:
df[df['text'] == ' ']

,text,label,source


In [16]:
df['label'].value_counts()

label
False    13181
True      9902
Name: count, dtype: int64

In [15]:
df['label'].groupby(df['source']).value_counts()

source          label
Vishvanews      False     909
africacheck     True     1046
                False     941
checkyourfact   False    1134
                True      234
eufactcheck     True       35
factcheck_afp   False    2942
                True        2
factograph      True       19
                False      12
factual_afp     False     449
                True        4
fatabyyano      False     290
                True       39
fullfact        True     1160
                False     422
politifact      False    3334
                True     2513
polygraph       False     174
                True       18
snopes          True     3874
                False    2295
truthorfiction  True      958
                False     279
Name: count, dtype: int64

In [19]:
mask = (df['source'] == 'Vishvanews')

In [32]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)

In [23]:
!pip install langdetect

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------- ----------------------------- 262.1/981.5 kB ? eta -:--:--
     -------------------- ----------------- 524.3/981.5 kB 1.1 MB/s eta 0:00:01
     ------------------------------ ------- 786.4/981.5 kB 1.2 MB/s eta 0:00:01
     -------------------------------------- 981.5/981.5 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993251 sha256=52f2281df9d41736639eaeadb73b2a0f5f0dcacbffcda5bbcae087ced6f9346b
  Stored in directory: c:\users\local\appdata\local\pip\cache\wheels\c1\67\88\e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


### potrzebne narzędzie do detekcji języków

In [30]:
from langdetect import detect

df['lang'] = df['text'].apply(detect)

In [34]:
df[mask]

,text,label,source,lang
9902,राजस्थान में एक शांतिदूत महात्मा गांधी की प्रतिमा को कुर्सी बनाकर बैठा है,False,Vishvanews,hi
9911,Amazon has begun presale of its own tokens,False,Vishvanews,en
9925,Government has issued any order against sharing political posts online,False,Vishvanews,en
9930,"এই মানুহ জন এক জটিল ৰোগত আক্ৰান্ত, এটা যন্ত্ৰৰ সহায় লৈ তেওঁ জীয়াই আছে । ডাক্তৰৰ মতে তেওঁৰ HEART TRANSPLANT'ৰ বাবে ৫০০০০০০০ টকাৰ প্ৰয়োজন",False,Vishvanews,bn
9931,لندن میں بھی رکشہ,False,Vishvanews,ur
...,...,...,...,...
22565,பாகிஸ்தான் ஜிந்தாபாத்’ கோஷம்; அமுல்யா இவரும் விவசாயிதான் சொன்னா நம்புங்கப்பா,False,Vishvanews,ta
22776,Drinking cold water after meals can cause cancer,False,Vishvanews,en
22914,Full rainbow seen in Gujarat,False,Vishvanews,en
22919,हमारे राष्ट्रपति! महामहिम!,False,Vishvanews,mr


### Bardzo tu jest bogato, zostawimy tylko 'en'

In [36]:
df['lang'].value_counts()

lang
en    21391
fr      478
ar      330
ur      209
mr       94
      ...  
fi        1
sl        1
lv        1
kn        1
ne        1
Name: count, Length: 35, dtype: int64

In [37]:
df = df[df['lang'] == 'en']

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21391 entries, 0 to 23081
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    21391 non-null  object
 1   label   21391 non-null  bool  
 2   source  21391 non-null  object
 3   lang    21391 non-null  object
dtypes: bool(1), object(3)
memory usage: 689.4+ KB


In [39]:
df['label'].groupby(df['source']).value_counts()

source          label
Vishvanews      False     312
africacheck     True     1036
                False     894
checkyourfact   False    1133
                True      234
eufactcheck     True       35
factcheck_afp   False    2845
                True        2
factual_afp     False       1
fullfact        True     1144
                False     417
politifact      False    3288
                True     2501
polygraph       False     171
                True       17
snopes          True     3858
                False    2286
truthorfiction  True      943
                False     274
Name: count, dtype: int64

##### niestety, po dokładnym ręcznym przeglądzie, trzeba usunąć sporo 'false' z 'africacheck',  bo zaczynają się od 'No, ' i są całe sformułowane jako ZAPRZECZENIE kwestionowanej wiadomości

In [ ]:
mask = (df['source'] == 'africacheck') & (df['text'].str.contains('No, '))
df[mask]

,text,label,source,lang
10131,"No, Bola Ahmed Tinubu, former governor of Nigeria’s Lagos state, is not dead",False,africacheck,en
10235,"Kenyan deputy presidential candidate Gachagua said ‘medicine men’ to be used in areas without clinics? No, quote made up",False,africacheck,en
10255,"No, Robert F Kennedy didn’t say mRNA Covid vaccines damage our genetic material – and they can’t",False,africacheck,en
10457,"No, Switzerland hasn’t banned AstraZeneca vaccine, but requested more data for approval",False,africacheck,en
10500,"No, Kenya’s deputy president has not been arrested, photo is from 2010",False,africacheck,en
...,...,...,...,...
22941,"No, photo shows corporal in Cuban dictator Fulgencio Batista’s army receiving last rites, not farmer",False,africacheck,en
22958,"No, guava leaf drink won’t cure infertility",False,africacheck,en
22966,"No, ‘after’ photo not of Zambian woman recovered from lung cancer",False,africacheck,en
23014,"No, former US president Obama didn’t warn Africans against coronavirus vaccines",False,africacheck,en


In [ ]:
df = df[~mask]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21054 entries, 0 to 23081
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    21054 non-null  object
 1   label   21054 non-null  bool  
 2   source  21054 non-null  object
 3   lang    21054 non-null  object
dtypes: bool(1), object(3)
memory usage: 678.5+ KB


In [42]:
df[df['source'] == 'factual_afp']

,text,label,source,lang
17116,Cristiano Ronaldo a inscrit son premier but avec Al-Nassr,False,factual_afp,en


In [43]:
df.drop([17116], axis=0, inplace=True)

In [44]:
df[df['source'] == 'factual_afp']

,text,label,source,lang


##### teksty z 'polygraph' również nie są wiadomościami, ale podsumowaniami, które od razu ZAPRZECZAJĄ lub WARTOŚCIUJĄ wiadomości

In [46]:
mask = (df['source'] == 'polygraph')
df = df[~mask]

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20865 entries, 0 to 23081
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    20865 non-null  object
 1   label   20865 non-null  bool  
 2   source  20865 non-null  object
 3   lang    20865 non-null  object
dtypes: bool(1), object(3)
memory usage: 672.4+ KB


In [49]:
df = df.reset_index(drop=True)
df

,text,label,source,lang
0,"While serving as U.S. Vice President, Henry A. Wallace wrote a piece warning Americans about the dangers of fascism.",True,snopes,en
1,A 1964 campaign ad for President Lyndon B. Johnson featured a purported Republican voter expressing concerns which eerily echoed threads of debate in the GOP in 2016.,True,snopes,en
2,Fifty Republican former national security officials issued an open letter affirming they would not vote for Donald Trump and deeming him dangerously unqualified.,True,snopes,en
3,Elon Musk deleted a tweet where he called the Mastodon social media platform Masterbatedone.,True,snopes,en
4,Texas Lt. Gov. Dan Patrick said COVID-19 social-distancing restrictions should be eased because there are more important things than living.,True,snopes,en
...,...,...,...,...
20860,A photograph shows Lev Parnas at a family birthday party with Donald Trump in 1990.,False,snopes,en
20861,BJP spokesperson Sambit Patra will anchor shows on NDTV,False,factcheck_afp,en
20862,Photo shows Queen Elizabeth II's corgis being brought back from Balmoral after her death,False,factcheck_afp,en
20863,Myanmar military owned TV channel announced a new law to nationalize extra oxen if any household owns more than two oxen.,False,factcheck_afp,en


In [52]:
label_map = {
    False: 1,
    True: 0
}

df["fake"] = df["label"].map(label_map)

In [53]:
df

,text,label,source,lang,fake
0,"While serving as U.S. Vice President, Henry A. Wallace wrote a piece warning Americans about the dangers of fascism.",True,snopes,en,0
1,A 1964 campaign ad for President Lyndon B. Johnson featured a purported Republican voter expressing concerns which eerily echoed threads of debate in the GOP in 2016.,True,snopes,en,0
2,Fifty Republican former national security officials issued an open letter affirming they would not vote for Donald Trump and deeming him dangerously unqualified.,True,snopes,en,0
3,Elon Musk deleted a tweet where he called the Mastodon social media platform Masterbatedone.,True,snopes,en,0
4,Texas Lt. Gov. Dan Patrick said COVID-19 social-distancing restrictions should be eased because there are more important things than living.,True,snopes,en,0
...,...,...,...,...,...
20860,A photograph shows Lev Parnas at a family birthday party with Donald Trump in 1990.,False,snopes,en,1
20861,BJP spokesperson Sambit Patra will anchor shows on NDTV,False,factcheck_afp,en,1
20862,Photo shows Queen Elizabeth II's corgis being brought back from Balmoral after her death,False,factcheck_afp,en,1
20863,Myanmar military owned TV channel announced a new law to nationalize extra oxen if any household owns more than two oxen.,False,factcheck_afp,en,1


In [54]:
df_for_export = df.drop(['label', 'source', 'lang'], axis=1)
df_for_export

,text,fake
0,"While serving as U.S. Vice President, Henry A. Wallace wrote a piece warning Americans about the dangers of fascism.",0
1,A 1964 campaign ad for President Lyndon B. Johnson featured a purported Republican voter expressing concerns which eerily echoed threads of debate in the GOP in 2016.,0
2,Fifty Republican former national security officials issued an open letter affirming they would not vote for Donald Trump and deeming him dangerously unqualified.,0
3,Elon Musk deleted a tweet where he called the Mastodon social media platform Masterbatedone.,0
4,Texas Lt. Gov. Dan Patrick said COVID-19 social-distancing restrictions should be eased because there are more important things than living.,0
...,...,...
20860,A photograph shows Lev Parnas at a family birthday party with Donald Trump in 1990.,1
20861,BJP spokesperson Sambit Patra will anchor shows on NDTV,1
20862,Photo shows Queen Elizabeth II's corgis being brought back from Balmoral after her death,1
20863,Myanmar military owned TV channel announced a new law to nationalize extra oxen if any household owns more than two oxen.,1


In [55]:
df_for_export['fake'].value_counts()

fake
1    11112
0     9753
Name: count, dtype: int64

In [56]:
df_for_export.to_csv('ClaimsKG_for_modeling.csv', index=False)